## Kernel Ridge Regression (KRR)
Regresión no-lineal regularizada.
* Generar conjunto de entrenamiento para predecir la función seno. 
* Expandir entradas en potencias (matriz de Vandermonde) para el ajuste polinómico con el regresor lineal (LR) y regresor lineal regularizado (LRR)
* Obtener pesos optimos para LR, LRR y KRR
* Calcula la salida para datos de test [0,2pi]
* Representa la salida estimada y la deseada

Analiza los resultados:
* Modifica la constante de regularización.
* Modifica la anchura del kernel RBF.
* Modifica el orden del ajuste polinómico. 

In [10]:
import numpy as np
# Genero datos de train
N=12
#X = np.linspace(0, np.pi * 2.0, N)
rng=np.random.RandomState(4)
X_train = rng.uniform(0, np.pi * 2.0, N)[:,np.newaxis]
y_train = np.sin( X_train )
print(X_train.shape[0])

12


In [14]:
%qtconsole

In [2]:
# Entreno modelos lineales

# Variable de entrada constante a uno para incluir bias en los pesos 
def addones(XX):
    return np.concatenate([XX, np.ones((XX.shape[0],1))],axis=1)
XX=X_train
M=5 #M<2 linear regression ; M>=2 polynomic regression
for p in range(2,M+1,1): XX = np.concatenate([X_train**p, XX],axis=1)
XX=addones(XX)
Y=y_train

# np.linalg.inv()
# XX.T.dot
# np.eye()

# Regresion lineal sin regularizacion
Wlr=XXX #pesos del modelo 

# Regresion lineal con regularizacion
l=0.001 #constante de regularizacion (lambda)
I=XXX  #matriz identidad
Wlrr=XXX #pesos del modelo 

NameError: name 'XXX' is not defined

In [ ]:
# Entreno molelo no lineal
from sklearn.metrics.pairwise import rbf_kernel
gamma=XXX #inversa de la anchura del kernel
K=XXX #matriz kernel de train
In=XXX  #matriz identidad
alpha=XXX #pesos del modelo 

In [ ]:
# Predicción
# Datos de test
X = np.linspace(0, np.pi * 2.0, 100)
X =X[:,np.newaxis]
# Señal deseada
y = np.sin( X )

#Predicciones
y_lr=XXX #prediccion lineal
y_lrr=XXX #prediccion lineal regularizado

Ktest=XXX #matriz kernel de test
y_krr=XXX #prediccion kernel ridge

In [ ]:
# Representación de la salida estimada
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(X, y, 'k-', label='deseada', lw=1)
plt.plot(X, y_lr, '--', color=(0.8, 0.2, 0.2), label='LR', lw=3)
plt.plot(X, y_lrr, '--', color=(0.2, 0.8, 0.2), label='LRR', lw=3)
plt.plot(X, y_krr, '--', color=(0.2, 0.2, 0.8), label='KRR', lw=3)
plt.plot(X_train, np.zeros((12,1)), 'o', color=(0.2, 0.2, 0.8), label='KRR', lw=3)

plt.xlim([0.0, X.max(0)])
plt.ylim([-1.05, 1.05])
plt.xlabel('entrada')
plt.ylabel('salida')
plt.title('Regresión')
plt.legend(loc="upper right")
